In [1]:
%%capture
import sys
import numpy as np

%pip install pyomo >/dev/null 2>/dev/null
%pip install highspy >/dev/null 2>/dev/null

solver = 'appsi_highs'

import pyomo.environ as pyo
SOLVER = pyo.SolverFactory(solver)

assert SOLVER.available(), f"Solver {solver} is not available."

import pandas as pd
import csv

In [2]:
#Intialize the model
model = pyo.ConcreteModel("National Park Road Trip")

In [3]:
#Import Data
activity_table = pd.read_excel("164 Mini Project Data .xlsx", sheet_name="Activities 2")
dist_matrix = pd.read_excel("164 Mini Project Data .xlsx", sheet_name="Distances 2")
dist_matrix = dist_matrix.drop(columns=["Unnamed: 0"])
time_matrix = pd.read_excel("164 Mini Project Data .xlsx", sheet_name="Travel Time") 
lodging_table = pd.read_excel("164 Mini Project Data .xlsx", sheet_name="Lodgings 2")
dist_matrix.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.0,4.8,2.9,3.8,3.8,28.5,29.3,61.7,28.3,28.6,47.6,200,15
1,4.8,0.0,7.7,1.8,1.0,33.3,34.1,66.5,33.1,33.4,52.4,200,15
2,2.9,7.7,0.0,6.7,6.7,27.0,27.8,60.2,27.9,27.2,46.2,200,20
3,3.8,1.8,6.7,0.0,0.6,32.4,33.1,65.5,32.2,32.5,51.5,200,15
4,3.8,1.0,6.7,0.6,0.0,32.3,33.0,65.4,32.0,32.4,51.4,200,10


In [4]:
#Sets
model.I = pyo.Set(initialize=activity_table['Activity'].dropna().tolist()) #set of activities
model.L = pyo.Set(initialize=lodging_table['Lodging'].dropna().tolist()) #set of lodging options
model.K = model.K = pyo.Set(initialize=[1,2,3]) #set of days
model.N = model.I | model.L #set of all nodes


In [5]:
#Parameters
distance_dict = {}
for i, src in enumerate(activity_table['Activity']):
    for j, dest in enumerate(activity_table['Activity']):
        if i != j:
            distance_dict[(src, dest)] = dist_matrix.iloc[i][j]
        else:
            distance_dict[(src, dest)] = 0


duration_dict = activity_table.set_index("Activity")["Duration"].to_dict() #duration of each activity
activityType_dict = activity_table.set_index("Activity")["Category"].to_dict() #type of each activity
cost_dict = activity_table.set_index("Activity")["Cost"].to_dict() #cost of each activity
funPoints_dict = activity_table.set_index("Activity")["Fun Points"].to_dict() #fun points of each activity
lodging_cost_dict = lodging_table.set_index("Lodging")["Cost ($/person)"].to_dict() #cost of each lodging option


model.distance = pyo.Param(model.N, model.N, initialize=distance_dict) #distance between each pair of activities
M = len(model.I) + 10
model.cost = pyo.Param(model.I, initialize=cost_dict) #cost of each activity
model.lodging_cost = pyo.Param(model.L, initialize=lodging_cost_dict) #cost of each lodging option

#


'Any'. The default domain for Param objects is 'Any'.  However, we will be
changing that default to 'Reals' in the future.  If you really intend the
domain of this Paramto be 'Any', you can suppress this warning by explicitly
specifying 'within=Any' to the Param constructor.  (deprecated in 5.6.9, will
be removed in (or after) 6.0) (called from
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-
packages/pyomo/core/base/indexed_component.py:718)


In [6]:
#Decision Variables
model.x = pyo.Var(model.I, model.K, within= pyo.Binary) #1 if activity i is selected on day k, 0 otherwise
model.y = pyo.Var(model.N, model.N, model. K, within= pyo.Binary) #1 if activity i is selected on day k, 0 otherwise
model.z_start = pyo.Var(model.L, model.K, within= pyo.Binary) #starting lodging option on day k
model.z_end = pyo.Var(model.L, model.K, within= pyo.Binary) #ending lodging option on day k
model.u = pyo.Var(model.I, model.K, within= pyo.NonNegativeIntegers, bounds = (0,M)) #Visiting Order

In [7]:
#objective function
def obj_rule(model):
    travel_cost = sum(model.distance[i,j] * model.y[i,j,k] for i in model.N for j in model.N for k in model.K if i != j)
    activity_cost = sum(model.cost[i] * model.x[i,k] for i in model.I for k in model.K)
    return travel_cost + activity_cost
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)


In [8]:
#Constraints

# #1. Each activity can only be selected once
# def activity_once_rule(model, i):
#     return sum(model.x[i,k] for k in model.K) <= 1
# model.activity_once = pyo.Constraint(model.I, rule=activity_once_rule)

# #Flow Constraints
# model.in_flow = pyo.Constraint(model.I, model.K, rule=lambda m, i, k: sum(m.y[j,i,k] for j in m.N if j != i) == m.x[i,k])
# model.out_flow = pyo.Constraint(model.I, model.K, rule=lambda m, i, k: sum(m.y[i,j,k] for j in m.N if j != i) == m.x[i,k])

# #Lodging Constraints per day
# def lodging_once_rule(model, k):
#     return sum(model.z[l,k] for l in model.L) == 1
# model.lodging_once = pyo.Constraint(model.K, rule=lodging_once_rule)



# #Day start and end
# model.lodging_start = pyo.Constraint(model.L, model.K, rule=lambda m, l, k: sum(m.y[l,j,k] for j in m.I if l != j) == m.z[l,k])
# model.lodging_end = pyo.Constraint(model.L, model.K, rule=lambda m, l, k: sum(m.y[j,l,k] for j in m.I if l != j) == m.z[l,k])

# #Subtour Elimination
# def subtour_elimination_rule(model, i, j, k):
#     if i != j:
#         return model.u[i,k] - model.u[j,k] + M * model.y[i,j,k] <= M - 1
#     else:
#         return pyo.Constraint.Skip
# model.subtour_elimination = pyo.Constraint(model.I, model.I, model.K, rule=subtour_elimination_rule)


# #Constraints on activities per day
# # def activity_per_day_rule(model, k):
# #     return sum(model.x[i,k] for i in model.I) <= 10
# # model.activity_per_day = pyo.Constraint(model.K, rule=activity_per_day_rule)




# Constraint: each activity at most once
model.activity_once = pyo.Constraint(model.I, rule=lambda m, i: sum(m.x[i,k] for k in m.K) <= 1)

# Flow constraints linking x and y
model.in_flow = pyo.Constraint(model.I, model.K, rule=lambda m, i, k: sum(m.y[j,i,k] for j in m.N if j != i) == m.x[i,k])
model.out_flow = pyo.Constraint(model.I, model.K, rule=lambda m, i, k: sum(m.y[i,j,k] for j in m.N if j != i) == m.x[i,k])

# Lodging start and end constraints per day
model.one_start_lodging_per_day = pyo.Constraint(model.K, rule=lambda m, k: sum(m.z_start[l,k] for l in m.L) == 1)
model.one_end_lodging_per_day = pyo.Constraint(model.K, rule=lambda m, k: sum(m.z_end[l,k] for l in m.L) == 1)

# Route must start and end at selected lodging
model.lodging_departure = pyo.Constraint(model.K, rule=lambda m, k: sum(m.y[l,j,k] for l in m.L for j in m.I if l != j) == 1)
model.lodging_arrival = pyo.Constraint(model.K, rule=lambda m, k: sum(m.y[i,l,k] for i in m.I for l in m.L if i != l) == 1)

# Start day 1 at LAX
model.start_at_LAX = pyo.Constraint(model.L, rule=lambda m, l: m.z_start[l,1] == 1 if l == 'LAX' else pyo.Constraint.Skip)

# Optional: Ensure if ending at l on day k, must start at l on day k+1
# Skipping day K to prevent index error
model.link_days = pyo.Constraint(model.L, model.K, rule=lambda m, l, k: m.z_end[l,k] == m.z_start[l,k+1] if k < max(m.K) else pyo.Constraint.Skip)

# Subtour elimination constraints (MTZ)
def subtour_rule(m, i, j, k):
    if i != j:
        return m.u[i,k] - m.u[j,k] + M * m.y[i,j,k] <= M - 1
    return pyo.Constraint.Skip
model.subtour_elim = pyo.Constraint(model.I, model.I, model.K, rule=subtour_rule)

In [9]:
#Solve the model
SOLVER.solve(model)
model.display()

RuntimeError: A feasible solution was not found, so no solution can be loaded. If using the appsi.solvers.Highs interface, you can set opt.config.load_solution=False. If using the environ.SolverFactory interface, you can set opt.solve(model, load_solutions = False). Then you can check results.termination_condition and results.best_feasible_objective before loading a solution.